In [1]:
import pandas as pd
import numpy as np

In [2]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [3]:
train["Age"] = train["Age"].fillna(train["Age"].median())

In [4]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

In [5]:
train.loc[train["Sex"] == "female", "Sex"] = 1
train.loc[train["Sex"] == "male", "Sex"] = 0

In [6]:
def h(x):
    return 1/(1+np.exp(-x))

def dj_dtheta(theta,X,Y,j):
    result=0
    for i in range(np.size(Y)):
        error=Y[i]-h(np.dot(theta,X[i]))
        result+=(error*X[i,j])
    return result

def grad_des(X,Y,theta):
    lnr=0.0001
    epoch=50000
    for i in range(epoch):
        if(i%5000==0):
            print(i,theta)
        new_theta=[]
        for j in range(X.shape[1]):
            a=theta[j]+lnr*dj_dtheta(theta,X,Y,j)
            new_theta.append(a)
        theta=np.array(new_theta)
        
    return theta
    
    

In [7]:
data = np.array(train[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
normalizer=max(data[:,[3]])
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]*data[:,[1]]
data = np.hstack([data, add_fea])
print(data)

[[1.    3.    0.    0.275 0.    0.825]
 [1.    1.    1.    0.475 1.    0.475]
 [1.    3.    1.    0.325 0.    0.975]
 ...
 [1.    3.    1.    0.35  0.    1.05 ]
 [1.    1.    0.    0.325 1.    0.325]
 [1.    3.    0.    0.4   2.    1.2  ]]


In [8]:
theta=np.array([0,0,0,0,0,0])
Y=np.array(train["Survived"].values, dtype = float)
theta=grad_des(data,Y,theta)
print("theta",theta)

0 [0 0 0 0 0 0]
5000 [ 1.0580518  -0.77391077  2.58049429 -0.39406555  0.31220009 -1.02262703]
10000 [ 1.20147915 -0.83176125  2.57898639 -0.71149562  0.31327821 -0.88695742]
15000 [ 1.31902074 -0.8801017   2.57787329 -0.97713368  0.31427267 -0.77072286]
20000 [ 1.41755183 -0.92060108  2.57711713 -1.19978408  0.31511667 -0.67348931]
25000 [ 1.50023352 -0.95456617  2.57660713 -1.38658616  0.3158316  -0.59205624]
30000 [ 1.56967274 -0.98307709  2.57626635 -1.54344656  0.31643655 -0.52377899]
35000 [ 1.62803263 -1.0070287   2.5760416  -1.67526325  0.31694807 -0.466476  ]
40000 [ 1.67711172 -1.027164    2.57589606 -1.78610645  0.31738037 -0.41834267]
45000 [ 1.71840826 -1.04410114  2.57580431 -1.87936528  0.31774559 -0.3778823 ]
theta [ 1.75317245 -1.05835537  2.57574879 -1.95786686  0.31805405 -0.34385066]


In [9]:
def isSurvived(theta,X,m):
    result=[]
    for i in range(m):
        ans=h(np.dot(theta,X[i]))
        if(ans>=0.5):
            ans=1
        else:
            ans=0
        result.append(ans)
    return result
        

dic=dict()
dic["PassengerId"]=np.array(test["PassengerId"].values, dtype = int)
test.loc[test["Sex"] == "female", "Sex"] = 1
test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test["Embarked"] == "C", "Embarked"] = 1
test.loc[test["Embarked"] == "Q", "Embarked"] = 2
data=np.array(test[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]*data[:,[1]]
data = np.hstack([data, add_fea])
dic["Survived"]=isSurvived(theta,data,418)
df=pd.DataFrame(dic)
df.to_csv("T12(ageXPclass).csv",index=False)








In [11]:
def accuracy(theta,X,Y):
    count=0
    for i in range(np.size(Y)):
        predY=h(np.dot(theta,X[i]))
        if(predY>=0.5):
            predY=1
        else:
            predY=0
        
        if(predY==Y[i]):
            count+=1
    print(count/np.size(Y))
    



data = np.array(train[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
normalizer=max(data[:,[3]])
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]*data[:,[1]]
data = np.hstack([data, add_fea])
Y=np.array(train["Survived"].values, dtype = float)
accuracy(theta,data,Y)
        

0.7901234567901234
